In [350]:
import pandas as pd
from sklearn import linear_model


In [351]:
df = pd.read_csv("SPY Historical Data.csv")

In [352]:
df.drop(df.columns[0], axis=1, inplace=True)

In [353]:
df

,Price,Open,High,Low,Vol.,Change %
0,442.13,442.49,442.79,441.37,41.22,-0.08%
1,442.49,442.19,442.91,441.81,46.93,0.17%
2,441.76,440.26,441.79,439.91,38.97,0.63%
3,438.98,439.78,440.39,438.74,46.73,-0.49%
4,441.15,438.53,441.17,436.12,58.05,0.81%
...,...,...,...,...,...,...
145,378.69,377.85,380.58,377.72,51.18,-0.67%
146,381.26,380.59,381.49,377.10,71.68,0.57%
147,379.10,376.10,379.90,375.91,68.77,1.49%
148,373.55,369.71,376.98,369.12,108.00,0.60%


In [354]:
# df = df[df.columns[4:5]]

In [355]:
df[df.columns[0:]] = df[df.columns[0:]].replace('[\%,]', '', regex=True).astype(float)
df

,Price,Open,High,Low,Vol.,Change %
0,442.13,442.49,442.79,441.37,41.22,-0.08
1,442.49,442.19,442.91,441.81,46.93,0.17
2,441.76,440.26,441.79,439.91,38.97,0.63
3,438.98,439.78,440.39,438.74,46.73,-0.49
4,441.15,438.53,441.17,436.12,58.05,0.81
...,...,...,...,...,...,...
145,378.69,377.85,380.58,377.72,51.18,-0.67
146,381.26,380.59,381.49,377.10,71.68,0.57
147,379.10,376.10,379.90,375.91,68.77,1.49
148,373.55,369.71,376.98,369.12,108.00,0.60


In [356]:
df[df.columns[0:]] = df[df.columns[0:]].replace('[\%,]', '', regex=True).astype(float)
df
def f(row):
    if row[5] > 0:
        return 1
    else:
        return 0

def floatToInt(row):
    return int(row[5]*100)

# df['changeInt'] = df.apply(floatToInt, axis=1)


In [357]:
df['signal'] = df.apply(f, axis=1)

In [358]:
df = df.reindex(index=df.index[::-1])

In [359]:
df['signal'] = df['signal'].shift(-1)

In [360]:
dff = df.drop(df.tail(7).index,inplace=False)

In [361]:
dff

,Price,Open,High,Low,Vol.,Change %,signal
149,371.33,368.10,372.50,368.05,66.43,0.69,1.0
148,373.55,369.71,376.98,369.12,108.00,0.60,1.0
147,379.10,376.10,379.90,375.91,68.77,1.49,1.0
146,381.26,380.59,381.49,377.10,71.68,0.57,0.0
145,378.69,377.85,380.58,377.72,51.18,-0.67,1.0
...,...,...,...,...,...,...,...
11,439.94,437.52,440.30,436.79,63.77,1.03,1.0
10,441.02,439.32,441.02,439.26,43.72,0.25,0.0
9,439.01,439.92,439.92,435.99,67.40,-0.46,0.0
8,438.83,439.65,440.27,437.42,52.47,-0.04,1.0


In [362]:
from sklearn import tree

clf = linear_model.LogisticRegression()
clf = clf.fit(dff[dff.columns[:len(dff.columns)-1]], dff['signal'])

In [365]:
clf.predict([[438.51,437.97,440.05,437.79,68.95,-0.49
]])

array([1.])

In [368]:
from sklearn import linear_model
cnt = 0
def backtest():
    cnt = 0
    for i in range(2, 100):
        modified_df = df.drop(df.tail(i).index,inplace=False) # drop last n rows
        actual = df.tail(i)
        actual = actual.head(1)
        clf = linear_model.LogisticRegression()
        clf = clf.fit(modified_df[modified_df.columns[:len(modified_df.columns)-1]], modified_df['signal'])
        x = clf.predict(actual[actual.columns[:len(actual.columns)-1]])
#         print(modified_df)
        print(actual)
        print(int(x[0]))
        if int(actual['signal']) == int(x[0]):
            cnt += 1
    return cnt

In [369]:
backtest()

    Price    Open    High     Low   Vol.  Change %  signal
1  442.49  442.19  442.91  441.81  46.93      0.17     0.0
1
    Price    Open    High     Low   Vol.  Change %  signal
2  441.76  440.26  441.79  439.91  38.97      0.63     1.0
1
    Price    Open    High     Low   Vol.  Change %  signal
3  438.98  439.78  440.39  438.74  46.73     -0.49     1.0
1
    Price    Open    High     Low   Vol.  Change %  signal
4  441.15  438.53  441.17  436.12  58.05      0.81     0.0
0
    Price   Open    High     Low   Vol.  Change %  signal
5  437.59  440.3  440.85  437.31  58.78     -0.21     1.0
1
    Price    Open    High     Low   Vol.  Change %  signal
6  438.51  437.97  440.05  437.79  68.95     -0.49     0.0
1
    Price   Open    High    Low   Vol.  Change %  signal
7  440.65  439.9  441.79  439.9  47.44      0.41     0.0
1
    Price    Open    High     Low   Vol.  Change %  signal
8  438.83  439.65  440.27  437.42  52.47     -0.04     1.0
1
    Price    Open    High     Low  Vol.  Chang

c:\users\iamsh\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\iamsh\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

     Price    Open    High     Low   Vol.  Change %  signal
16  431.34  436.01  436.05  430.93  75.87     -0.78     0.0
1
     Price    Open    High     Low   Vol.  Change %  signal
17  434.75  434.92  435.52  432.75  55.13     -0.34     0.0
1
     Price    Open   High     Low   Vol.  Change %  signal
18  436.24  437.45  437.9  434.92  64.13      0.15     0.0
1
     Price    Open    High     Low   Vol.  Change %  signal
19  435.59  436.28  437.83  435.32  52.91     -0.34     1.0
1
     Price    Open    High     Low   Vol.  Change %  signal
20  437.08  435.48  437.25  434.98  52.89      0.36     0.0
1
     Price    Open    High     Low   Vol.  Change %  signal
21  435.52  432.59  435.77  432.33  76.24      1.07     1.0
1
     Price    Open    High     Low  Vol.  Change %  signal
22  430.92  428.75  431.72  427.58  97.6     -0.81     1.0
1
     Price    Open    High     Low   Vol.  Change %  signal
23  434.46  433.68  434.75  431.52  63.55      0.35     0.0
1


c:\users\iamsh\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\iamsh\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

     Price    Open    High     Low   Vol.  Change %  signal
24  432.93  433.72  433.95  430.01  68.71     -0.18     1.0
0
     Price    Open   High     Low  Vol.  Change %  signal
25  433.72  431.67  434.1  430.52  57.7      0.76     0.0
1
     Price    Open    High     Low   Vol.  Change %  signal
26  430.43  428.93  430.53  428.82  53.44      0.55     1.0
1
     Price    Open   High     Low   Vol.  Change %  signal
27  428.06  427.33  428.7  427.28  64.83      0.08     1.0
1
    Price    Open    High     Low   Vol.  Change %  signal
28  427.7  427.88  428.56  427.13  35.97      0.05     1.0
1
     Price    Open    High     Low   Vol.  Change %  signal
29  427.47  427.21  427.64  425.89  53.16       0.2     1.0
1
     Price    Open    High     Low   Vol.  Change %  signal
30  426.61  426.05  427.07  425.57  58.13      0.36     1.0
1
    Price    Open    High     Low   Vol.  Change %  signal
31  425.1  424.94  425.55  424.63  45.11      0.59     1.0
1


...

     Price    Open    High     Low   Vol.  Change %  signal
79  417.26  417.25  417.91  415.73  82.04      0.33     0.0
1
     Price    Open    High     Low   Vol.  Change %  signal
80  415.87  413.74  416.16  413.69  60.23      1.07     1.0
1
     Price    Open    High     Low   Vol.  Change %  signal
81  411.45  412.83  413.96  410.87  61.66     -0.34     1.0
1
     Price    Open    High     Low   Vol.  Change %  signal
82  412.86  411.53  413.53  411.12  56.55       0.3     0.0
1
     Price    Open    High    Low  Vol.  Change %  signal
83  411.64  410.85  411.93  410.2  56.7      0.04     1.0
1
     Price    Open    High     Low  Vol.  Change %  signal
84  411.49  408.39  411.67  408.26  61.1      0.73     1.0
1
     Price    Open    High     Low   Vol.  Change %  signal
85  408.52  407.93  408.58  406.93  57.86      0.47     1.0
1
     Price    Open    High     Low   Vol.  Change %  signal
86  406.59  405.94  406.96  405.45  55.84      0.12     1.0
1
     Price    Open    High    

c:\users\iamsh\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\iamsh\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

     Price    Open   High     Low    Vol.  Change %  signal
90  396.33  395.34  398.0  395.31  112.73      0.41     1.0
0
     Price    Open    High     Low   Vol.  Change %  signal
91  394.73  394.42  395.45  393.02  76.26     -0.27     1.0
1
     Price   Open    High     Low    Vol.  Change %  signal
92  395.78  394.4  396.75  392.81  108.11     -0.05     0.0
1
     Price    Open    High     Low    Vol.  Change %  signal
93  395.98  390.93  396.41  390.29  114.41      1.61     0.0
1
    Price    Open    High    Low    Vol.  Change %  signal
94  389.7  385.98  390.55  383.9  116.13      0.56     1.0
1
     Price   Open    High     Low   Vol.  Change %  signal
95  387.52  391.0  392.75  387.47  97.59     -0.51     1.0
1
    Price    Open    High     Low   Vol.  Change %  signal
96  389.5  391.91  393.46  388.66  90.69     -0.79     0.0
1
     Price    Open    High     Low   Vol.  Change %  signal
97  392.59  390.03  394.07  389.97  73.78       0.8     0.0
1
     Price    Open    High  

c:\users\iamsh\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\iamsh\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

50